In [ ]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sklearn
#import scikeras
import tensorflow

print("scikit-learn version:", sklearn.__version__)
#print("scikeras version:", scikeras.__version__)
print("tensorflow version:", tensorflow.__version__)


scikit-learn version: 1.5.2
tensorflow version: 2.18.0


In [ ]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1


In [ ]:
pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 103.0 MB/s eta 0:00:00


In [ ]:
import os
os._exit(00)

In [ ]:
!pip install scikeras

In [ ]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from tensorflow.keras.layers import Dense, Flatten, LSTM, Bidirectional, Input, Dropout, InputLayer, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [ ]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [ ]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [ ]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [ ]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [ ]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [ ]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

In [ ]:
train_size = int(len(df_META) * 0.75)
test_size = len(df_META) - train_size

# META

## Train Test

In [ ]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [ ]:
X_META = train_META.drop(columns=["Stockprice_META","date","sentiment_META","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

#Normalized

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_META = scaler.fit_transform(X_META)

In [ ]:
X_META.shape[0]

564

In [ ]:
look_back = 10

X_META1, y_META1 = [], []
for i in range(len(X_META) - look_back):
    X_META1.append(X_META[i:i + look_back])
    y_META1.append(y_META[i + look_back])

X_META1, y_META1 = np.array(X_META1), np.array(y_META1)

In [ ]:
y_META1.shape

(554,)

In [ ]:
X_META1.shape

(554, 10, 10)

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

#MODEL

In [ ]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(SimpleRNN(256, input_shape=(look_back, X_META1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_META1, y_META1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_META1)
mse = mean_squared_error(y_META1, y_pred)
rmse = root_mean_squared_error(y_META1, y_pred)
mae = mean_absolute_error(y_META1, y_pred)
mape = mean_absolute_percentage_error(y_META1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 150, 'optimizer': 'RMSprop'}
Test Set Mean Squared Error: 225.3094
Test Set Root Mean Squared Error: 15.0103
Test Set Mean Absolute Error: 10.7688
Test Set Mean Absolute Percentage Error: 0.0453


#APPL

In [ ]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","sentiment_AAPL", "date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [ ]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [ ]:
look_back = 10

X_AAPL1, y_AAPL1 = [], []
for i in range(len(X_AAPL) - look_back):
    X_AAPL1.append(X_AAPL[i:i + look_back])
    y_AAPL1.append(y_AAPL[i + look_back])

X_AAPL1, y_AAPL1 = np.array(X_AAPL1), np.array(y_AAPL1)

In [ ]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(SimpleRNN(256, input_shape=(look_back, X_AAPL1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150]}



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_AAPL1, y_AAPL1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_AAPL1)
mse = mean_squared_error(y_AAPL1, y_pred)
rmse = root_mean_squared_error(y_AAPL1, y_pred)
mae = mean_absolute_error(y_AAPL1, y_pred)
mape = mean_absolute_percentage_error(y_AAPL1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 150, 'optimizer': 'RMSprop'}
Test Set Mean Squared Error: 38.0283
Test Set Root Mean Squared Error: 6.1667
Test Set Mean Absolute Error: 4.8129
Test Set Mean Absolute Percentage Error: 0.0285


#MSFT

In [ ]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","sentiment_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [ ]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [ ]:
look_back = 10

X_MSFT1, y_MSFT1 = [], []
for i in range(len(X_MSFT) - look_back):
    X_MSFT1.append(X_MSFT[i:i + look_back])
    y_MSFT1.append(y_MSFT[i + look_back])

X_MSFT1, y_MSFT1 = np.array(X_MSFT1), np.array(y_MSFT1)

In [ ]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(SimpleRNN(256, input_shape=(look_back, X_MSFT1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_MSFT1, y_MSFT1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_MSFT1)
mse = mean_squared_error(y_MSFT1, y_pred)
rmse = root_mean_squared_error(y_MSFT1, y_pred)
mae = mean_absolute_error(y_MSFT1, y_pred)
mape = mean_absolute_percentage_error(y_MSFT1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 100, 'optimizer': 'Nadam'}
Test Set Mean Squared Error: 114.7716
Test Set Root Mean Squared Error: 10.7132
Test Set Mean Absolute Error: 8.5262
Test Set Mean Absolute Percentage Error: 0.0288


#NVDA

In [ ]:
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_NVDA = train_NVDA.drop(columns=["Stockprice_NVDA","sentiment_NVDA","date", "Interest","RETAIL_SALES", "US_Dollar_Index"])
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [ ]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [ ]:
look_back = 10

X_NVDA1, y_NVDA1 = [], []
for i in range(len(X_NVDA) - look_back):
    X_NVDA1.append(X_NVDA[i:i + look_back])
    y_NVDA1.append(y_NVDA[i + look_back])

X_NVDA1, y_NVDA1 = np.array(X_NVDA1), np.array(y_NVDA1)

In [ ]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(SimpleRNN(256, input_shape=(look_back, X_NVDA1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)

param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_NVDA1, y_NVDA1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_NVDA1)
mse = mean_squared_error(y_NVDA1, y_pred)
rmse = root_mean_squared_error(y_NVDA1, y_pred)
mae = mean_absolute_error(y_NVDA1, y_pred)
mape = mean_absolute_percentage_error(y_NVDA1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 150, 'optimizer': 'Adam'}
Test Set Mean Squared Error: 8.6319
Test Set Root Mean Squared Error: 2.9380
Test Set Mean Absolute Error: 2.0147
Test Set Mean Absolute Percentage Error: 0.0558


#GOOG

In [ ]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL", "sentiment_GOOGL", "date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [ ]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [ ]:
look_back = 10

X_GOOG1, y_GOOG1 = [], []
for i in range(len(X_GOOG) - look_back):
    X_GOOG1.append(X_GOOG[i:i + look_back])
    y_GOOG1.append(y_GOOG[i + look_back])

X_GOOG1, y_GOOG1 = np.array(X_GOOG1), np.array(y_GOOG1)

In [ ]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(SimpleRNN(256, input_shape=(look_back, X_GOOG1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_GOOG1, y_GOOG1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_


y_pred = best_model.predict(X_GOOG1)
mse = mean_squared_error(y_GOOG1, y_pred)
rmse = root_mean_squared_error(y_GOOG1, y_pred)
mae = mean_absolute_error(y_GOOG1, y_pred)
mape = mean_absolute_percentage_error(y_GOOG1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'optimizer': 'Nadam'}
Test Set Mean Squared Error: 57.8307
Test Set Root Mean Squared Error: 7.6046
Test Set Mean Absolute Error: 5.7755
Test Set Mean Absolute Percentage Error: 0.0484
